# Imports

In [146]:
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset

In [37]:
# Load the California house dataset
data = pd.read_csv('housing.csv')

# Features

We need to keep these features:
- 'latitude'
- 'longitude'
- 'housing_median_age'
- 'total_rooms'
- 'total_bedrooms'
- 'population'
- 'households'
- 'median_income'
- 'median_house_value'
- 'ocean_proximity'

In [38]:
features = [
    "latitude",
    "longitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income",
    "median_house_value",
    "ocean_proximity",
]

In [39]:
subset_data = data[features]

# Data preparation

## Fill in the missing values with median

In [40]:
subset_data.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [41]:
# Calculate the median of total_bedrooms
median_bedrooms = subset_data["total_bedrooms"].median()

In [42]:
median_bedrooms

435.0

In [43]:
subset_data = subset_data.fillna(median_bedrooms)

In [44]:
subset_data.isnull().sum()

latitude              0
longitude             0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

## Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.

In [45]:
subset_data['rooms_per_household'] = subset_data['total_rooms']/subset_data['households']

In [46]:
subset_data['rooms_per_household']

0        6.984127
1        6.238137
2        8.288136
3        5.817352
4        6.281853
           ...   
20635    5.045455
20636    6.114035
20637    5.205543
20638    5.329513
20639    5.254717
Name: rooms_per_household, Length: 20640, dtype: float64

## Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.

In [47]:
subset_data['bedrooms_per_room'] = subset_data['total_bedrooms']/subset_data['total_rooms']

In [48]:
subset_data['bedrooms_per_room']

0        0.146591
1        0.155797
2        0.129516
3        0.184458
4        0.172096
           ...   
20635    0.224625
20636    0.215208
20637    0.215173
20638    0.219892
20639    0.221185
Name: bedrooms_per_room, Length: 20640, dtype: float64

## Create a new column population_per_household by dividing the column population by the column households from dataframe.

In [49]:
subset_data['population_per_household'] = subset_data['population']/subset_data['households']

In [50]:
subset_data['population_per_household']

0        2.555556
1        2.109842
2        2.802260
3        2.547945
4        2.181467
           ...   
20635    2.560606
20636    3.122807
20637    2.325635
20638    2.123209
20639    2.616981
Name: population_per_household, Length: 20640, dtype: float64

# Question 1

What is the most frequent observation (mode) for the column ocean_proximity?

In [51]:
subset_data['ocean_proximity'].mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

# Question 2

Create the correlation matrix for the numerical features of your train dataset.
- In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

In [52]:
# Select numerical features of the dataset
subset_data.dtypes

latitude                    float64
longitude                   float64
housing_median_age          float64
total_rooms                 float64
total_bedrooms              float64
population                  float64
households                  float64
median_income               float64
median_house_value          float64
ocean_proximity              object
rooms_per_household         float64
bedrooms_per_room           float64
population_per_household    float64
dtype: object

In [53]:
numerical_features = subset_data.columns.values.tolist()


In [54]:
numerical_features

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [55]:
corr_matrix = subset_data.corr()

C:\Users\Melanie\AppData\Local\Temp\ipykernel_28108\455652441.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = subset_data.corr()


In [56]:
corr_matrix

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,bedrooms_per_room,population_per_household
latitude,1.000000,-0.924664,0.011173,-0.036100,-0.066484,-0.108785,-0.071035,-0.079809,-0.144160,0.106389,-0.098619,0.002366
longitude,-0.924664,1.000000,-0.108197,0.044568,0.069120,0.099773,0.055310,-0.015176,-0.045967,-0.027540,0.081205,0.002476
housing_median_age,0.011173,-0.108197,1.000000,-0.361262,-0.319026,-0.296244,-0.302916,-0.119034,0.105623,-0.153277,0.135622,0.013191
total_rooms,-0.036100,0.044568,-0.361262,1.000000,0.927058,0.857126,0.918484,0.198050,0.134153,0.133798,-0.187381,-0.024581
total_bedrooms,-0.066484,0.069120,-0.319026,0.927058,1.000000,0.873535,0.974366,-0.007617,0.049457,0.001765,0.071649,-0.028325
population,-0.108785,0.099773,-0.296244,0.857126,0.873535,1.000000,0.907222,0.004834,-0.024650,-0.072213,0.010035,0.069863
households,-0.071035,0.055310,-0.302916,0.918484,0.974366,0.907222,1.000000,0.013033,0.065843,-0.080598,0.034498,-0.027309
median_income,-0.079809,-0.015176,-0.119034,0.198050,-0.007617,0.004834,0.013033,1.000000,0.688075,0.326895,-0.545298,0.018766
median_house_value,-0.144160,-0.045967,0.105623,0.134153,0.049457,-0.024650,0.065843,0.688075,1.000000,0.151948,-0.233303,-0.023737
rooms_per_household,0.106389,-0.027540,-0.153277,0.133798,0.001765,-0.072213,-0.080598,0.326895,0.151948,1.000000,-0.370308,-0.004852


What are the two features that have the biggest correlation in this dataset?

In [57]:
# Extract the correlation coefficients from the correlation matrix (I use the absolute value to not miss any correlation)
ordered_correlations = abs(corr_matrix).unstack().sort_values(ascending=False)
ordered_correlations

latitude                  latitude                    1.000000
longitude                 longitude                   1.000000
bedrooms_per_room         bedrooms_per_room           1.000000
rooms_per_household       rooms_per_household         1.000000
median_house_value        median_house_value          1.000000
                                                        ...   
longitude                 population_per_household    0.002476
latitude                  population_per_household    0.002366
population_per_household  latitude                    0.002366
total_bedrooms            rooms_per_household         0.001765
rooms_per_household       total_bedrooms              0.001765
Length: 144, dtype: float64

In [58]:
print(ordered_correlations[ordered_correlations < 1])

households                total_bedrooms              0.974366
total_bedrooms            households                  0.974366
                          total_rooms                 0.927058
total_rooms               total_bedrooms              0.927058
longitude                 latitude                    0.924664
                                                        ...   
                          population_per_household    0.002476
latitude                  population_per_household    0.002366
population_per_household  latitude                    0.002366
total_bedrooms            rooms_per_household         0.001765
rooms_per_household       total_bedrooms              0.001765
Length: 132, dtype: float64


The two variables 'total_bedrooms' and 'households' have the greatest correlation between them.

# Make median_house_value binary

We need to turn the median_house_value variable from numeric into binary.

Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.

In [59]:
# Calculate the mean of 'median_house_value'
median_house_value_mean = subset_data['median_house_value'].mean()
median_house_value_mean

206855.81690891474

In [60]:
subset_data['above_average'] = np.where(subset_data['median_house_value'] > median_house_value_mean, 1, 0)

# Split the data

Split your data in train/val/test sets, with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value (median_house_value) is not in your dataframe.

In [136]:
# First generate the full training dataset and the test dataset
full_train, test = train_test_split(subset_data, test_size=0.2, random_state=42)

In [137]:
# Then separate the full training dataset into train and validation datasets. We have to adjust the proportions:
# we need 20% of the total number of rows from the 80% of values, which means we are looking for 0.25 proportion for the validation dataset
train, val = train_test_split(full_train, test_size=0.25, random_state=42)

In [92]:
# I remove the 'median_house_value' from the three datasets and keep above_average as y values (I do not remove it yet)
y_train = train['above_average']
y_val = val['above_average']
y_test = test['above_average']

del train['median_house_value'] 
del val['median_house_value'] 
del test['median_house_value'] 

# Question 3

Calculate the mutual information score between above_average and ocean_proximity . Use the training set only.

Round it to 2 decimals using round(score, 2)

What is their mutual information score?

In [89]:
round(mutual_info_score(train['above_average'], train['ocean_proximity']),2)

0.1

# Question 4

Now let's train a logistic regression

Remember that we have one categorical variable ocean_proximity in the data. Include it using one-hot encoding.

Fit the model on the training dataset.

To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [93]:
# Remove the 'above_average' column from the features datasets
del train['above_average'] 
del val['above_average'] 
del test['above_average'] 

In [94]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

In [95]:
# One hot encoding of 'ocean_proximity' categorical variable using DictVectorizer
train_dicts = train.to_dict(orient='records')
train_dicts[0]

{'latitude': 34.43,
 'longitude': -119.67,
 'housing_median_age': 39.0,
 'total_rooms': 1467.0,
 'total_bedrooms': 381.0,
 'population': 1404.0,
 'households': 374.0,
 'median_income': 2.3681,
 'ocean_proximity': '<1H OCEAN',
 'rooms_per_household': 3.9224598930481283,
 'bedrooms_per_room': 0.25971370143149286,
 'population_per_household': 3.7540106951871657}

In [96]:
# Instantiate DictVectorizer
dv = DictVectorizer(sparse=False)

In [97]:
# Fit it and transform our training dataset
X_train = dv.fit_transform(train_dicts)

In [98]:
# Check the shape of our feature matrix
print(X_train.shape, train.shape)

(12384, 16) (12384, 12)


Four columns have been added to our dataset by one-hot encoding the ocean_proximity variable.

In [99]:
X_train

array([[2.59713701e-01, 3.74000000e+02, 3.90000000e+01, ...,
        3.92245989e+00, 3.81000000e+02, 1.46700000e+03],
       [1.30227981e-01, 8.06000000e+02, 2.40000000e+01, ...,
        7.56451613e+00, 7.94000000e+02, 6.09700000e+03],
       [2.34624146e-01, 3.37000000e+02, 4.10000000e+01, ...,
        3.90801187e+00, 3.09000000e+02, 1.31700000e+03],
       ...,
       [1.82879377e-01, 6.02000000e+02, 1.80000000e+01, ...,
        5.54983389e+00, 6.11000000e+02, 3.34100000e+03],
       [2.29126214e-01, 3.50000000e+02, 1.60000000e+01, ...,
        4.41428571e+00, 3.54000000e+02, 1.54500000e+03],
       [2.09574468e-01, 2.15000000e+02, 3.50000000e+01, ...,
        4.37209302e+00, 1.97000000e+02, 9.40000000e+02]])

In [100]:
# Train the logistic regression model with this new feature matrix
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [101]:
# Check the accuracy on the train dataset
score = model.score(X_train, y_train)
print("Training Accuracy Score", round(score,2))

Training Accuracy Score 0.83


In [102]:
# Prepare X_val as I prepared X_train:
val_dicts = val.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_val = dv.fit_transform(val_dicts)

In [103]:
# Calculate the accuracy on the validation dataset
score = model.score(X_val, y_val)
print("Validation Accuracy Score", round(score,2))

Validation Accuracy Score 0.84


# Question 5

Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?

total_rooms

total_bedrooms

population

households

In [120]:
def logistic_regression_accuracy(feature_list, train_df, val_df, test_df, y_train, y_val, y_test):
    
    # Clearly state the features used
    print(f"The features used are {feature_list}.")
    
    # Keep the feature list in our feature dataframes
    train_features = train_df[feature_list]
    val_features = val_df[feature_list]
    test_features = test_df[feature_list]
    
    # Create the corresponding feature matrices
    ## Train
    train_dicts = train_features.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train_features = dv.fit_transform(train_dicts)
    ## Val
    val_dicts = val_features.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_val_features = dv.fit_transform(val_dicts)
    ## Test
    test_dicts = test_features.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_test_features = dv.fit_transform(test_dicts)
    
    # Train the model with these features:
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_features, y_train)
    
    # Accuracy on the train dataset
    score_train = model.score(X_train_features, y_train)
    print("Training Accuracy Score", round(score_train,3))
    # Accuracy on the validation set
    score_val = model.score(X_val_features, y_val)
    print("Validation Accuracy Score", round(score_val,3))
    # Accuracy on the test set
    score_test = model.score(X_test_features, y_test)
    print("Test Accuracy Score", round(score_test,3))
    
    return score_train, score_val, score_test

In [121]:
features_to_keep = ['total_rooms', 'total_bedrooms', 'population', 'households']

In [122]:
score_train_all, score_val_all, score_test_all = logistic_regression_accuracy(features_to_keep,
                                                                             train,
                                                                             val,
                                                                             test,
                                                                             y_train,
                                                                             y_val,
                                                                             y_test)

The features used are ['total_rooms', 'total_bedrooms', 'population', 'households'].
Training Accuracy Score 0.698
Validation Accuracy Score 0.71
Test Accuracy Score 0.704


In [123]:
features_without_total_rooms = ['total_bedrooms', 'population', 'households']

In [124]:
score_train_all, score_val_all, score_test_all = logistic_regression_accuracy(features_without_total_rooms,
                                                                             train,
                                                                             val,
                                                                             test,
                                                                             y_train,
                                                                             y_val,
                                                                             y_test)

The features used are ['total_bedrooms', 'population', 'households'].
Training Accuracy Score 0.621
Validation Accuracy Score 0.628
Test Accuracy Score 0.622


In [125]:
features_without_total_bedrooms = ['total_rooms', 'population', 'households']

In [126]:
score_train_all, score_val_all, score_test_all = logistic_regression_accuracy(features_without_total_bedrooms,
                                                                             train,
                                                                             val,
                                                                             test,
                                                                             y_train,
                                                                             y_val,
                                                                             y_test)

The features used are ['total_rooms', 'population', 'households'].
Training Accuracy Score 0.653
Validation Accuracy Score 0.661
Test Accuracy Score 0.655


In [127]:
features_without_population = ['total_rooms', 'total_bedrooms','households']

In [128]:
score_train_all, score_val_all, score_test_all = logistic_regression_accuracy(features_without_population,
                                                                             train,
                                                                             val,
                                                                             test,
                                                                             y_train,
                                                                             y_val,
                                                                             y_test)

The features used are ['total_rooms', 'total_bedrooms', 'households'].
Training Accuracy Score 0.647
Validation Accuracy Score 0.657
Test Accuracy Score 0.654


In [129]:
features_without_households = ['total_rooms', 'total_bedrooms', 'population']

In [130]:
score_train_all, score_val_all, score_test_all = logistic_regression_accuracy(features_without_households,
                                                                             train,
                                                                             val,
                                                                             test,
                                                                             y_train,
                                                                             y_val,
                                                                             y_test)

The features used are ['total_rooms', 'total_bedrooms', 'population'].
Training Accuracy Score 0.667
Validation Accuracy Score 0.672
Test Accuracy Score 0.675


In [131]:
# Calculate the difference in accuracy for the validation dataset for each feature compared to all features:
diff_total_rooms = 0.71-0.628
diff_total_bedrooms = 0.71-0.661
diff_population = 0.71-0.657
diff_households = 0.71-0.672

In [133]:
# Print the differences:
print(f"The accuracy difference between all features and total_rooms is: {round(diff_total_rooms,2)}.")
print(f"The accuracy difference between all features and total_bedrooms is: {round(diff_total_bedrooms,2)}.")
print(f"The accuracy difference between all features and households is: {round(diff_households,2)}.")
print(f"The accuracy difference between all features and population is: {round(diff_population,2)}.")

The accuracy difference between all features and total_rooms is: 0.08.
The accuracy difference between all features and total_bedrooms is: 0.05.
The accuracy difference between all features and households is: 0.04.
The accuracy difference between all features and population is: 0.05.


The "households" variable presents the smallest difference compared to all features.

# Question 6

For this question, we'll see how to use a linear regression model from Scikit-Learn

We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.

Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.

This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]

Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

In [138]:
# I remove the 'median_house_value' from the three datasets and keep above_average as y values (I do not remove it yet)
y_train = train['median_house_value']
y_val = val['median_house_value']
y_test = test['median_house_value']

del train['median_house_value'] 
del val['median_house_value'] 
del test['median_house_value'] 

# Also delete the 'above_average' column (data leakage)
del train['above_average'] 
del val['above_average'] 
del test['above_average'] 

In [141]:
# Apply logarithmic transformation to the column
y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)

In [143]:
# Prepare the dataset
def prepare_feature_matrices(train,val,test):
    
    train_dicts = train.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    
    val_dicts = val.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_val = dv.fit_transform(val_dicts)
    
    test_dicts = test.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_test = dv.fit_transform(test_dicts)
    
    return X_train, X_val, X_test

In [144]:
# Generate feature matrices
X_train, X_val, X_test = prepare_feature_matrices(train, val, test)

In [148]:
def generate_rmse(y_pred, y_true):
    
    error = y_pred-y_true
    mse = (error**2).mean()
    rmse = np.sqrt(mse)
    
    return rmse

In [157]:
# Function to get the RMSE values on the validation dataset with Ridge trained on train dataset
def rmse_validation_ridge(X_train, X_val, y_train, y_val, alpha):
    
    model = Ridge(alpha=alpha, solver="sag", random_state=42)
    model.fit(X_train, y_train)
    
    y_pred_val = model.predict(X_val)
    rmse_val = generate_rmse(y_pred_val,y_val)
    
    print(f"The obtained RMSE on the validation dataset with alpha={alpha} is {round(rmse_val,3)}.")
    
    return round(rmse_val,3)

In [158]:
# Fit a Ridge regression model on the training dataset with the list of alpha parameters
rmse_val_list = []
alpha_list = [0, 0.01, 0.1, 1, 10]

for alpha in alpha_list:
    rmse_value = rmse_validation_ridge(X_train, X_val, y_train, y_val, alpha)
    rmse_val_list.append(rmse_value)

C:\Users\Melanie\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The obtained RMSE on the validation dataset with alpha=0 is 0.523.


C:\Users\Melanie\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The obtained RMSE on the validation dataset with alpha=0.01 is 0.523.


C:\Users\Melanie\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The obtained RMSE on the validation dataset with alpha=0.1 is 0.523.


C:\Users\Melanie\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The obtained RMSE on the validation dataset with alpha=1 is 0.523.
The obtained RMSE on the validation dataset with alpha=10 is 0.523.


C:\Users\Melanie\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [159]:
rmse_val_list

[0.523, 0.523, 0.523, 0.523, 0.523]

The smallest RMSE value is obtained with alpha=0.